In [85]:
import re
import os
from tika import parser
from gensim.parsing.preprocessing import remove_stopwords
import pandas as pd

#create dataframe for letter contents
files = [i for i in os.listdir("SequoiaLetters2")]
letters=[]
f_letters=[]
for file in files:
    filename = os.path.basename(file)
    letterName = filename.strip('.pdf')
    file_data = parser.from_file(file) 
    text = file_data['content']
    #Remove excess words (before and after letter)
    split_text = text.split("Disclosures", 1)
    text = split_text[0]
    split_text2 = text.split("Dear", 1)
    text = split_text2[1]
    filtered_text = remove_stopwords(text)
    letters.append([letterName,text])
    f_letters.append([letterName,filtered_text])

df= pd.DataFrame(letters, columns=['title','content'])
df = df.sort_values('title')
print(df)

      title                                            content
10  2016_Q2   Clients and Sequoia Shareholders: \n \nYou ma...
11  2016_Q3   Clients and Shareholders: \n\n \n\nThe third ...
15  2016_Q4   Shareholder: \n\nWe completed our most diffic...
2   2017_Q1   Shareholder: \n \nSequoia Fund generated a 5....
1   2017_Q2   Sequoia Shareholders and Clients: \n\nSequoia...
0   2017_Q3   Sequoia Shareholders and Clients:  \n\nSequoi...
5   2017_Q4   Sequoia Shareholders and Clients: \n\nSequoia...
14  2018_Q1   Sequoia Shareholders and Clients:  \n\nFor th...
12  2018_Q2   Sequoia Shareholders and Clients:  \n\nFor th...
13  2018_Q3   Sequoia Shareholders and Clients:  \n\nFor th...
8   2018_Q4   Sequoia Shareholders and Clients:  \n\nSequoi...
4   2019_Q1   Sequoia Shareholders and Clients:  \n\nFor th...
7   2019_Q2   Sequoia Shareholders and Clients:  \n\nFor th...
6   2019_Q3   Sequoia Shareholders and Clients:  \n\nFor th...
3   2019_Q4   Sequoia Shareholders and Clients:  \n\nSe

In [93]:
import spacy
nlp=spacy.load('en_core_web_sm')

letter = open("baupost1999.txt","r+")  

content = letter.read() 
content = content.replace('\n',' ')
content = content.replace('\'','')

doc=nlp(content)
for ent in doc.ents:
    if (ent.label_ == 'ORG'):
        print(ent.text,ent.label_)


Pactiv Corporation ORG
Tenneco Automotive ORG
Walker ORG
Tenneco Automotive ORG
Neiman Marcus ORG
Chemfirst ORG
LNR Corporation ORG
Octel ORG
Great Lakes Chemical Corporation ORG
TEL ORG
Octel ORG
TEL ORG
Stewart Enterprises ORG
Ballard Power Systems ORG
Saab ORG
Saab ORG
Celsius ORG
Saab ORG
Trustor Corporation ORG
Maxwell Communications ORG
Maxwell ORG
The Baupost Fund ORG


In [101]:
ner=nlp.get_pipe("ner")

# Training examples in the required format
TRAIN_DATA =[ ("Ucar is the world's leading manufacturer of graphite electrodes", {"entities": [(0, 3, "ORG")]}),
              ("Chargeurs is a French company which processes and trades in wool and produces fabrics", {"entities": [(0, 9, "ORG")]}),
              ("Lambert Fenchurch is a publicly traded insurance broker in the U.K", {"entities": [(0,17, "ORG"),(63,65,"LOC")]}),
              ("We added to our long position in Dentsply Sirona", {"entities": [(33,47, "ORG")]}),
              ("We added to our position in Carmax", {"entities": [(28,34, "ORG")]}),
              ("We trimmed our investment in Alphabet", {"entities": [(29,37, "ORG")]}),
              ("We bought shares of Amazon", {"entities": [(20,26, "ORG")]}),
              ("We purchased Berkshire stocks", {"entities": [(13,22, "ORG")]}),
              ("Naspers is a holding company located in South Africa", {"entities": [(0,7, "ORG"),(40,52,"LOC")]}),
              ("Its principal asset is a large stake in Tencent", {"entities": [(40,47, "ORG")]}),
              #("Tencent operates China's dominant social media platform", {"entities": [(0,7, "ORG")]}),
              ("A large interest in U.S cable company Charter Communications", {"entities": [(38,60, "ORG"),(20,23,"LOC")]}),
              ("We exited our position in Microsoft", {"entities": [(26,35, "ORG")]}),
              ("We cut our position in Jacobs Engineering", {"entities": [(23,41, "ORG")]})]
# Adding labels to the `ner`
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])
        

In [102]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path

# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(50):

    # shuufling examples  before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
        print("Losses", losses)

Losses {'ner': 3.1704290667788904e-06}
Losses {'ner': 2.033846933192586}
Losses {'ner': 2.0340778371316515}
Losses {'ner': 2.034077839253342}
Losses {'ner': 0.00018190128271990195}
Losses {'ner': 0.00018305146507113925}
Losses {'ner': 0.00019507853219536238}
Losses {'ner': 0.00020651616904710033}
Losses {'ner': 0.00021260403033221482}
Losses {'ner': 0.00021483141756100706}
Losses {'ner': 0.00021567282210104997}
Losses {'ner': 0.0006591434526057683}
Losses {'ner': 0.0016065232175669613}
Losses {'ner': 0.0016088005264955057}
Losses {'ner': 5.091428886667613}
Losses {'ner': 5.091489950034973}
Losses {'ner': 0.0009759435376211586}
Losses {'ner': 1.757259243389824}
Losses {'ner': 1.7572675549886243}
Losses {'ner': 1.7572675738315935}
Losses {'ner': 2.1962500502958893e-07}
Losses {'ner': 3.7182980167581835e-07}
Losses {'ner': 1.9997570639528794}
Losses {'ner': 1.999757064241544}
Losses {'ner': 6.256064245951931e-07}
Losses {'ner': 9.411749165333383e-05}
Losses {'ner': 0.011349928430455236}
L

In [105]:
doc=nlp(df.content[4])
for ent in doc.ents:
    if (ent.label_ == 'ORG'):
        print(ent.text,ent.label_)
        

Sequoia Shareholders ORG
Clients ORG
Sequoia Fund ORG
Material ORG
Electronic Arts ORG
Koninklijke Vopak ORG
Vopak ORG
While ORG
Though ORG
Reversing ORG
Rather ORG
The Fund’s 1-year ORG
5-year ORG
March 31 ORG
Current ORG
Performance ORG
DST Systems ORG
57th Street ORG
Suite ORG
New York ORG
NY ORG
Tel ORG
Ruane Cunniff ORG
Please ORG
Because ORG
December 31 ORG
Investor Day ORG
Friday ORG
May 17 ORG
10:00 ORG
the Grand Ballroom ORG
Plaza ORG
Hotel ORG
Ruane Cunniff ORG
Sincerely ORG
The Ruane Cunniff Investment Committee ORG
Arman Gokgol ORG
Kline ORG
Trevor Magyar ORG
Chase Sheridan ORG
Suite ORG
New York ORG
NY ORG
Tel ORG


In [82]:
nlp2=spacy.load("en_core_web_sm") 

# Getting the ner component
ner=nlp.get_pipe('ner')

LABEL = "SECTOR"
TRAIN_DATA =[ ("Ucar is the world's leading manufacturer of graphite electrodes", {"entities": [(43, 63, "SECTOR")]}),
              ("Chargeurs processes and trades in wool and produces fabrics", {"entities": [(52, 59, "SECTOR")]}),
              ("Lambert Fenchurch is a publicly traded insurance broker in the U.K", {"entities": [(39,48, "SECTOR")]}),
              ("Tencent operates China's dominant social media platform", {"entities": [(34,46, "SECTOR")]}),
              ("A large interest in U.S cable company Charter Communications", {"entities": [(23,28, "SECTOR")]}),
              ("Salesforce is a SaaS company", {"entities": [(16,20, "SECTOR")]})]

